# Activity
Whether it's a short data processing app, and api layer, or a full blown distributed system, many systems follow an activity paradigm. Once you see it, you can't unsee it:
1) An agent, or a process, works on some data
2) There is a temporary or ephemeral nature about "its" data. Like a player, an instance, a stock market algorithm, or even an api call. But within that moment, that local information in critical.
3) The ephemeral data should pass something on to the global context
4) The global context, objects, rankings, accounts, trades, need to change
5) The world updates, and with a new timestep, we expect notifications about all the worlds activity to flow
6) The agent has views, and filters / searches the worlds information

This loop is in many systems. Over time I have learned that much of my development activity is centered around "scaling" this exact loop:

1) in the smallest case, I write a one file simple client / server system.
2) Then i code up a front end, and I need to host it
3) Suddenly I need to scale the system across a million different nodes.

Moving from 1-2 is hard. It's about the same difficulty to move from 2-3. BUT, approaching every problem with (3) will increase the time of every project. If only there was a thin framework that one could adopt at (1) that would allow you to scale to (3) if that ever becomes needed.

Welcome to activity. Activity is the least amount of infastructre needed at (1) to ensure your application can seamlessly scale to (3) with minimal adjustments. It enforces patterns of object orientation (encapsulation), and statless communication via an event-subscriber model. But, thankfully, it mostly works using function decorators in python. This means, the (1) version can be as simple as a few function calls

Activity is built on processingNetwork, a generic processing engine that simplifies functional programming (making it feel procedural, if you squint at it.)

If the script you are working on will never be (3) then it's safe to say you should just code it. If it's worth 20 minutes of planning, then refactor onto activity right away, and you will be treated later,

In [1]:


# Agent
# ---
# Strategy (Strategy)
# Action Tracker (Order Tracker)

# System Template
# ----------------
# Local Environment (Account Log)
# World Environment (Broker View)

# Pace
# ---------
# TimeEvent
# Clock

# View
# -----------------
# TimeseriesEvent (MarketDataEmitter / Order Assignment Event)
# Filter (Filter Model)



In [7]:
import sys
sys.path.append('../../')
from flex import flex
from processingNetwork import ProcessingNode,ProcessingNetwork,QuickInputNode,QuickProcessingNode

def EnvironmentNode(feature,context):
    return "<"+feature+">"

def PrintNode(feature,context):
    print ('------------')
    print (feature)
    print ('------------')
    return str("{"+feature['input1']+" | "+feature['input2']+"}")

nodes = flex()
nodes['layer1'] = QuickInputNode(EnvironmentNode,('data',))
nodes['layer2a'] = QuickInputNode(EnvironmentNode,('layer1',))
nodes['layer2b'] = QuickInputNode(EnvironmentNode,('layer1',))
nodes['layer3'] = QuickProcessingNode(PrintNode,{'input1':('layer2a',),'input2':('layer2b',),})

pn = ProcessingNetwork(nodes)

# Result
Ok, so a function call. Big deal right? But, as cloud developers who want to create really big complex systems, we want to be able to "just write a function" and "scale to the cloud" in the same breath. Here is how we manage our ProcessingNetwork.

In [8]:
import pprint
# Process a result
data = pn.process({'data':'brian'})
pprint.pprint(data['layer3'])

------------
{'name': 'layer3', 'input1': '<<brian>>', 'input2': '<<brian>>'}
------------
'{<<brian>> | <<brian>>}'


In [9]:
# Wait, how was this made?
pprint.pprint(data)

{'data': 'brian',
 'env': 'Hey brian I did some work!',
 'env2': 'Hey Hey brian I did some work! I did some work!'}


In [10]:
# oh! cool. Also, what network made this?
pprint.pprint(pn.getNetworkTemplate())

{'env': {'dependencies': {'input': ('data',)},
         'instance': <__main__.EnvironmentNode object at 0x7f8c982d3048>,
         'name': 'env',
         'type': <class '__main__.EnvironmentNode'>},
 'env2': {'dependencies': {'input': ('env',)},
          'instance': <__main__.EnvironmentNode object at 0x7f8c982d3080>,
          'name': 'env2',
          'type': <class '__main__.EnvironmentNode'>}}


This is *almost* serializable. In fact, if we got rid of these class refrences, it would be possible to serialize everything about this experiment. To support serialization, the code can't live in a notebook -- we need to give it a home. The syntax for the serializable version is below, as an example: